## Fine-tuning
- This notebook contains code for the fine-tuning of target-task regressor using weights of pre-trained language model 
- The code is adapted from https://github.com/XinhaoLi74/MolPMoFiT/blob/master/notebooks/04_QSAR_Regression.ipynb

#### Install RDKit on Google colaboratory

In [ ]:
%%bash
add-apt-repository ppa:ubuntu-toolchain-r/test
apt-get update --fix-missing
apt-get dist-upgrade
wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
chmod +x Miniconda3-latest-Linux-x86_64.sh
./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
conda config --set always_yes yes --set changeps1 no
conda install -q -y -c conda-forge python=3.7
conda install -q -y -c conda-forge rdkit

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

Import the important libraries

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit import RDLogger 
RDLogger.DisableLog('rdApp.*') # switch off RDKit warning messages

from sklearn.model_selection import train_test_split

from fastai import *
from fastai.text import *
from fastai.vision import *

import numpy as np
import threading
import random
from sklearn.utils import shuffle 

Set the seed value

In [ ]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda: 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

# Data
Mount Google Drive to Google Colab to access the google drive files 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Create a path to save the results

data_path = Path('/content/gdrive/My Drive/results')
name = 'regressor'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)

In [ ]:
data = pd.read_csv('/content/gdrive/My Drive/data/740-reactions-yield.csv')
print('Dataset:', data.shape)

Dataset: (740, 2)


### Target task regressor fine-tuning on target task LM

Train-validation-test splits

- Split the data into train-validation-test sets
- Validation set is used for hyperparameter tuning
- Test set is used for the final performance evaluation

In [ ]:
random_seed(1234, True)

train_ , test = train_test_split(data, test_size=0.20, random_state=0)
train, valid = train_test_split(train_, test_size=0.108, random_state=0)

print(train.shape)
print(test.shape)
print(valid.shape)

(738, 2)
(2, 2)


### SMILES augmentation for regression task

- For the regression task, a gaussian noise (with mean zero and standard deviation, σg_noise) is added to the labels of the augmented SMILES during the training
- The number of augmented SMILES and σg_noise is tuned on the validation set

In [ ]:
def randomize_smiles(smiles):
    m = Chem.MolFromSmiles(smiles)
    ans = list(range(m.GetNumAtoms()))
    np.random.shuffle(ans)
    nm = Chem.RenumberAtoms(m,ans)
    return Chem.MolToSmiles(nm, canonical=False, isomericSmiles=True, kekuleSmiles=False)

In [ ]:
def ee_smiles_augmentation(df, N_rounds, noise):
    '''
    noise: add gaussion noise to the label
    '''
    dist_aug = {col_name: [] for col_name in df}

    for i in range(df.shape[0]):
        for j in range(N_rounds):
            dist_aug['smiles'].append(randomize_smiles(df.iloc[i].smiles))
            dist_aug['yield'].append(df.iloc[i]['yield'] + np.random.normal(0,noise))
    df_aug = pd.DataFrame.from_dict(dist_aug)
    df_aug = df_aug.append(df, ignore_index=True)
    return df_aug.drop_duplicates('smiles')

In [ ]:
random_seed(1234, True)

train_aug = ee_smiles_augmentation(train, 100, noise=0.0)
print("Train_aug: ", train_aug.shape)

Train_aug:  (74531, 2)


### Data pre-processing

Define a custom tokenizer

In [ ]:
# Don't include the defalut specific token of fastai, only keep the padding token
BOS,EOS,FLD,UNK,PAD = 'xxbos','xxeos','xxfld','xxunk','xxpad'
TK_MAJ,TK_UP,TK_REP,TK_WREP = 'xxmaj','xxup','xxrep','xxwrep'
defaults.text_spec_tok = [PAD]

special_tokens = ['[BOS]', '[C@H]', '[C@@H]','[C@]', '[C@@]','[C-]','[C+]', '[c-]', '[c+]','[cH-]',
                   '[nH]', '[N+]', '[N-]', '[n+]', '[n-]' '[NH+]', '[NH2+]',
                   '[O-]', '[S+]', '[s+]', '[S-]', '[O+]', '[SH]', '[B-]','[BH2-]', '[BH3-]','[b-]',
                   '[PH]','[P+]', '[I+]', 
                  '[Si]','[SiH2]', '[Se]','[SeH]', '[se]', '[Se+]', '[se+]','[te]','[te+]', '[Te]']

class MolTokenizer(BaseTokenizer):
    def __init__(self, lang = 'en', special_tokens = special_tokens):
        self.lang = lang
        self.special_tokens = special_tokens
        
    def tokenizer(self, smiles):
        # add specific token '[BOS]' to represetences the start of SMILES
        smiles = '[BOS]' + smiles
        regex = '(\[[^\[\]]{1,10}\])'
        char_list = re.split(regex, smiles)
        tokens = []
        
        if self.special_tokens:
            for char in char_list:
                if char.startswith('['):
                    if char in special_tokens:
                        tokens.append(str(char))
                    else:
                        tokens.append('[UNK]')
                else:
                    chars = [unit for unit in char]
                    [tokens.append(i) for i in chars]                    
        
        if not self.special_tokens:
            for char in char_list:
                if char.startswith('['):
                    tokens.append(str(char))
                else:
                    chars = [unit for unit in char]
                    [tokens.append(i) for i in chars]
                
        #fix the 'Br' be splited into 'B' and 'r'
        if 'B' in tokens:
            for index, tok in enumerate(tokens):
                if tok == 'B':
                    if index < len(tokens)-1: # make sure 'B' is not the last character
                        if tokens[index+1] == 'r':
                            tokens[index: index+2] = [reduce(lambda i, j: i + j, tokens[index : index+2])]
        
        #fix the 'Cl' be splited into 'C' and 'l'
        if 'l' in tokens:
            for index, tok in enumerate(tokens):
                if tok == 'l':
                    if tokens[index-1] == 'C':
                            tokens[index-1: index+1] = [reduce(lambda i, j: i + j, tokens[index-1 : index+1])]
        return tokens    
    
    def add_special_cases(self, toks):
        pass

In [ ]:
bs = 128
tok = Tokenizer(partial(MolTokenizer, special_tokens = special_tokens), n_cpus=6, pre_rules=[], post_rules=[])

Adpot the encoder of the pre-trained LM according to the target dataset



In [ ]:
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning) 
random_seed(1234, True)

lm_vocab = TextLMDataBunch.from_df(path, train_aug, valid, bs=bs, tokenizer=tok, 
                              chunksize=50000, text_cols=0, label_cols=1, max_vocab=60000, include_bos=False, min_freq=1, num_workers=0)
print(f'Vocab Size: {len(lm_vocab.vocab.itos)}')

Vocab Size: 32


In [ ]:
pretrained_model_path = Path('/content/gdrive/My Drive/results/MSPM/models')
pretrained_fnames = ['pre-trained_wt', 'pre-trained_vocab']
fnames = [pretrained_model_path/f'{fn}.{ext}' for fn,ext in zip(pretrained_fnames, ['pth', 'pkl'])]

In [ ]:
random_seed(1234, True)

lm_learner = language_model_learner(lm_vocab, AWD_LSTM, drop_mult=0.2, pretrained=False)
lm_learner = lm_learner.load_pretrained(*fnames)
lm_learner.freeze()
lm_learner.save_encoder(f'lm_encoder')

Create a text databunch for regression:

- It takes as input the train and validation data
- Pass the vocab of the pre-trained LM as defined in the previous step
- Specify the column containing text data and output
- Define the batch size according to the GPU memory available


In [ ]:
random_seed(1234, True)

data_clas = TextClasDataBunch.from_df(path, train_aug, valid, bs=bs, tokenizer=tok, 
                                          chunksize=50000, text_cols='smiles',label_cols='yield', 
                                          vocab=lm_vocab.vocab, max_vocab=60000, include_bos=False, min_freq=1, num_workers=0)

print(f'Vocab Size: {len(data_clas.vocab.itos)}')

Vocab Size: 32


### Training the regression model

Create a learner for regression:

- Pass the databunch
- Load the encoder of the pre-trained LM
- The drop_mult hyperparameter can be tuned
- The model is evaluated using RMSE and R-squared value as error metric

In [ ]:
random_seed(1234, True)

reg_learner = text_classifier_learner(data_clas, AWD_LSTM, pretrained=False, drop_mult=0.0, metrics = [r2_score, rmse])
reg_learner.load_encoder(f'lm_encoder')
reg_learner.freeze()

In [ ]:
# Model architecture
reg_learner.model

The regressor is fine-tuned all at once without any frozen weights (i.e., no gradual unfreezing)

In [ ]:
random_seed(1234, True)

reg_learner.unfreeze()
reg_learner.fit_one_cycle(25, 1e-3, moms=(0.8,0.7))

Save the trained learner. It is then later used for prediction on test set

In [ ]:
split_id = 0
reg_learner.save(f'{split_id}_reg')

#### Evaluation on the Test Set

In [ ]:
def test_smiles_augmentation(df, N_rounds):
    dist_aug = {col_name: [] for col_name in df}

    for i in range(df.shape[0]):
        for j in range(N_rounds):
            dist_aug['smiles'].append(randomize_smiles(df.iloc[i].smiles))
            dist_aug['yield'].append(df.iloc[i]['yield'])
    df_aug = pd.DataFrame.from_dict(dist_aug)
    
    return pd.DataFrame.from_dict(dist_aug)

In [ ]:
random_seed(1234, True)

tok_new = TokenizeProcessor(tokenizer=tok, chunksize=50000, include_bos=False)
num_new = NumericalizeProcessor(vocab=lm_vocab.vocab, max_vocab=60000, min_freq=1) 

ee_data = pd.read_csv('/content/gdrive/My Drive/data/740-reactions-yield.csv')

train_ , test_ = train_test_split(ee_data, test_size=0.2, random_state=0)
train , test = train_test_split(ee_data, test_size=0.2, random_state= 0)

train_.insert(2, 'valid', False)
test_.insert(2, 'valid', True)
df = pd.concat([train_, test_])

In [ ]:
random_seed(1234, True)

preds = []

# Randomized SMILES Predictions
for i in range(4):
    np.random.seed(12*i)
    test_aug = test_smiles_augmentation(test, 1)
    
    #model
    
    test_aug.insert(2, 'valid', True)
    df_aug = pd.concat([train, test_aug])
    test_db = (TextList.from_df(df_aug, path, cols='smiles', processor=[tok_new, num_new]).split_from_df(col='valid').label_from_df(cols='yield', label_cls=FloatList).databunch(bs=128))
    
    learner = text_classifier_learner(test_db, AWD_LSTM, config=None, pretrained=False, drop_mult=0.0, metrics = [r2_score, rmse])
    
    learner.load(f'{split_id}_reg'); 
  
    #get predictions
    pred,lbl = learner.get_preds(ordered=True)
    
    preds.append(pred)

# Canonical SMILES Predictions

test_db = (TextList.from_df(df, path, cols='smiles', processor=[tok_new, num_new]).split_from_df(col='valid').label_from_df(cols='yield', label_cls=FloatList).databunch(bs=128))

learner = text_classifier_learner(test_db, AWD_LSTM, config=None, pretrained=False, drop_mult=0.0, metrics = [r2_score, rmse])

learner.load(f'{split_id}_reg');


#get predictions
pred_canonical,lbl = learner.get_preds(ordered=True)
    
preds.append(pred_canonical)

The test set performance is evaluated using the predictions based on the canonical SMILES as well as that employing test-time augmentation

In [ ]:
print('Test Set (Canonical)')
print('RMSE:', root_mean_squared_error(pred_canonical,lbl))
print('MAE:', mean_absolute_error(pred_canonical,lbl))
print('R2:', r2_score(pred_canonical,lbl))

In [ ]:
avg_preds = sum(preds)/len(preds)
#print('\n')
print('Test Set (Average)')
print('RMSE:', root_mean_squared_error(avg_preds,lbl))
print('MAE:', mean_absolute_error(avg_preds,lbl))
print('R2:', r2_score(avg_preds,lbl))